This tutorial can be downloaded [link](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/west_300.ipynb).

# 3.0 Parallelization

As detailed in the [1.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/doc/tutorials/west_100.ipynb), the GW workflow of WEST involves three steps: 

- Step 1: Ground state `pw.x`
- Step 2: Screening `wstat.x`
- Step 3: Quasiparticle corrections `wfreq.x`

In each step, several levels of [MPI](https://en.wikipedia.org/wiki/Message_Passing_Interface) and [OpenMP](https://www.openmp.org) parallelization may be utilized to accelerate the calculation, by distributing the workload across processors. In this tutorial, we will explore the parallelization levels available in WEST.

WEST adopts a multilevel parallelization strategy:

- `world`: The group of all MPI processes.
- `image`: `world` can be partitioned into `images`. When using the projective dielectric eigendecomposition (PDEP) technique to compute the static dielectric screening, perturbations are distributed across images. Each image is responsible for computing the density response only for the perturbations owned by the image.
- `pool`: Each `image` can be partitioned into `pools`, each working on a subgroup of k-points and spin channels.
- `bgrp`: Each `pool` can be partitioned into `band groups`, each working on a subgroup of bands (wavefunctions).
- `R&G`: Wavefunctions in the plane-wave basis set, as well as density in either real (R) or reciprocal (G) space, are distributed across MPI processes. Fast Fourier transform (FFT) is used to transform quantities between R space and G space. By default, a 3D FFT is parallelized by distributing planes of the 3D grid in real space to MPI processes. In reciprocal space, the G-vectors are distributed.
- `SIMT`: Each MPI process in a band group is apable of offloading computations to a GPU device using the single instruction, multiple threads (SIMT) protocol.

![Image of parallelization levels](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/images/parallelization.jpg)

In this figure, the multilevel parallelization of WEST is exemplified for the case of 16 total MPI processes. The processes are divided into two images. Each image is divided into two pools, each of which is further divided into two band groups. Within each band group there are two MPI processes, each of which is connected to a GPU device. Using GPUs to accelerate WEST calculations is discussed in the [4.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/west_400.ipynb).

`wstat.x` and `wfreq.x` support image parallelization (activated by command line option `-nimage` or `-ni`), pool parallelization (activated by `-npool` or `-nk`) for spin channels but currectly not for k-points, band parallelization (activated by `-nband` or `-nb`), and R&G parallelization (activated by default). Some remarks:

- If none of these options is specified, WEST by default only uses the R&G level of parallelization.
- The R&G level helps reduce memory footprint (per MPI process), as each process only stores a fraction of the wavefunctions. The bgrp level reduces memory usage to some extent by distributing the bands to MPI processes for some but not all quantities. The image and pool level do not reduce memory usage, as the wavefunctions (and some other quantities) are replicated for each image and pool, i.e., only the computations are distributed, not the data.
- The amount of data exchange between images and pools is low. Data exchange between band groups is mild. Data exchange within a band group is significant, as the parallel FFT algorithm is known to require all-to-all communications, that is, every MPI process must exchange data with every other process. Therefore, putting too many MPI processes into the R&G level may not improve performance at all.

For more information about the implementation, the reader is referred to [Govoni et al., J. Chem. Theory Comput. 11, 2680 (2015)](https://doi.org/10.1021/ct500958p) and [Yu et al., J. Chem. Theory Comput. 18, 4690-4707 (2022)](https://doi.org/10.1021/acs.jctc.2c00241).

Now let us try running WEST with different parallelization strategies. We will reuse the input files used in the [1.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/west_100.ipynb). Download these files to your current directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/silane/pw.in
wget -N -q http://www.west-code.org/doc/training/silane/wstat.in
wget -N -q http://www.west-code.org/doc/training/silane/wfreq.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.2.upf

First we perform a ground state calculation using `pw.x` as usual.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

The [1.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/doc/tutorials/west_100.ipynb) runs `wstat.x` and `wfreq.x` with two MPI processes (CPU cores).

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

Since none of `-nimage`, `-npools`, `-nband` is specified, WEST (`wstat.x` and `wfreq.x`) defaults to the R&G level of parallelization. Alternatively, the calculation can be carried out using image parallelization, which is controled by the `-ni` command line switch.

In [ ]:
%%bash
mpirun -n 2 wstat.x -ni 2 -i wstat.in > wstat.2image.out
mpirun -n 2 wfreq.x -ni 2 -i wfreq.in > wfreq.2image.out

The reader is encouraged to try this out and compare the output files obtained with and without `-ni 2`.

- Using a different parallelization strategy should not change the physical observables except for very minor numerical deviations.
- The estimated memory consumption roughly doubles when using `-ni 2`. As already explained, when using the image parallelization, the computations are distributed across images, while the wavefunctions are replicated for each image. 
- Parallelization should show a small impact on the time to solution for small calculations like this.

In addition to image parallelization, WEST supports parallelization over spin and band indices. In the current version of WEST, parallelization over k-points is not yet implemented. The parallelization levels can be used separately or collaboratively. We underline the importance of choosing the parallelization strategy for running large-scale simulations on massively parallel supercomputers. The optimal parameters would of course depend on the specifics of the simulation and the hardware. In general, the image parallelization level is preferred over the other parallelization levels. Using the spin parallelization level usually speeds up calculations of spin-polarized systems. The efficiency of the R&G level of parallelization is largely limited by the efficiency of parallel FFTs. We therefore do not recommend putting more than a few hundred MPI processes into this level.

Below are some restrictions and guidelines when choosing the parallelization parameters. `n_total` denotes the total number of MPI processes. `n_image`, `n_pool`, and `n_bgrp` denote the numbers of images, pools, and band groups, respectively. `nbndocc` denotes the number of occupied bands. `n_pdep_eigen`, `n_pdep_eigen_use`, and `qp_bandrange` are described in the [manual](http://www.west-code.org/doc/West/v5.0.0/manual.html#manual).

`wstat.x`:

- n_image <= n_pdep_eigen \[**required**\]
- n_bgrp <= nbndocc \[**required**\]
- n_total / (n_image * n_pool * n_bgrp) < 1000 \[**recommended** for FFT efficiency on CPUs\]
- n_total / (n_image * n_pool * n_bgrp) as small as possibe \[**recommended** for FFT efficiency on GPUs\]

`wfreq.x`:

- n_image <= n_pdep_eigen_use \[**required**\]
- n_bgrp <= nbndocc \[**required**\]
- n_bgrp <= qp_bandrange[2] - qp_bandrange[1] + 1 \[**required**\]
- n_total / (n_image * n_pool * n_bgrp) < 1000 \[**recommended** for FFT efficiency on CPUs\]
- n_total / (n_image * n_pool * n_bgrp) as small as possible \[**recommended** for FFT efficiency on GPUs\]
- n_pdep_eigen_use / n_image > 32 \[**recommended** for GEMM efficiency\]

An example of using WEST with all the parallelization levels and GPUs is given in the [4.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/west_400.ipynb).